# Caso completo sin AutoML

## El problema: Customer Churn

![](https://chartmogul.com/blog/wp-content/uploads/2022/02/blogWhat_s-a-good-Customer-Churn-Rate-1200x500.jpeg)

La rotación de clientes se define como cuando los clientes o suscriptores dejan de hacer negocios con una empresa o servicio.

Cada fila representa un cliente, cada columna contiene los atributos del cliente.

El conjunto de datos incluye información sobre:

- Clientes que se fueron en el último mes: la columna se llama **Churn**
- Servicios a los que se ha suscrito cada cliente: teléfono, líneas múltiples, Internet, seguridad en línea, respaldo en línea, protección de dispositivos, soporte técnico y transmisión de TV y películas.
- Información de la cuenta del cliente: cuánto tiempo ha sido cliente, contrato, método de pago, facturación electrónica, cargos mensuales y cargos totales
- Información demográfica sobre los clientes: género, rango de edad y si tienen parejas y dependientes

## Comprender el contexto empresarial y el problema

Después de esas reuniones tenemos que comprobar los datos existentes en la empresa y encontrar información útil en ellos. Supongamos que lo hicimos y después de un proceso de integración de datos creamos un conjunto de datos completo para nuestros clientes y su información. Recuerda que estamos trabajando con una empresa de telecomunicaciones.

## Librerías

In [ ]:
!pip install datatable

In [ ]:
import pandas as pd
import datatable as dt
from datatable import f
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import scipy.stats as stats
import warnings
import numpy as np
warnings.filterwarnings("ignore")

## Cargar datos

In [ ]:
df = dt.fread("../data/churn_data.csv")

In [ ]:
df.head()

In [ ]:
df.shape

El color significa el tipo de datos donde el rojo denota cadena, el verde denota int, el amarillo significa booleano y el azul significa flotador.

## ¿Cuántos clientes se han ido?

In [ ]:
df[f.Churn == "Yes", dt.count()]

In [ ]:
1869/7043

Se han ido 1869 clientes, eso significa el 26% de nuestros clientes. Entonces, si recordamos las métricas del negocio, tenemos un problema.

## ¿Cuánto dinero hemos perdido por la pérdida de clientes?

In [ ]:
df[:, dt.count(), dt.by(dt.f.Churn)]

In [ ]:
df[dt.f.Churn == 'Yes', 'TotalCharges'].sum1()

Hemos perdido $2.862.926 debido a la rotación de clientes. Así que vamos a tratar de resolver este problema.

## EDA

In [ ]:
df_pandas = df.to_pandas()

In [ ]:
df_pandas.head()

In [ ]:
def diagnostic_plots(df, variable):
    
    plt.figure(figsize=(20, 9))

    plt.subplot(1, 3, 1)
    sns.distplot(df[variable], bins=30, kde=True)
    plt.title('Histogram')
    
    plt.subplot(1, 3, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)
    plt.ylabel('RM quantiles')

    plt.subplot(1, 3, 3)
    sns.boxplot(x=df[variable])
    plt.title('Boxplot')
    
    plt.show()

In [ ]:
num_columns=df_pandas.select_dtypes(include=["number"]).columns
num_columns

In [ ]:
for i in num_columns:
    diagnostic_plots(df_pandas,i)

In [ ]:
sns.pairplot(df_pandas.drop("SeniorCitizen",axis=1),hue="Churn",aspect=3);

In [ ]:
fig = px.histogram(df_pandas, x="Churn")
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

In [ ]:
fig = px.histogram(df_pandas, x="Churn", color="SeniorCitizen")
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

In [ ]:
fig = px.histogram(df_pandas, x="Churn", color="OnlineSecurity", barmode="group")
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

In [ ]:
fig = px.box(df_pandas, x='Churn', y = 'tenure')
fig.show()

In [ ]:
ax = sns.kdeplot(df_pandas.MonthlyCharges[(df_pandas["Churn"] == 'No') ],
                color="Red", shade = True)
ax = sns.kdeplot(df_pandas.MonthlyCharges[(df_pandas["Churn"] == 'Yes') ],
                ax =ax, color="Blue", shade= True)
ax.legend(["Not Churn","Churn"],loc='upper right')
ax.set_ylabel('Density')
ax.set_xlabel('Monthly Charges')
ax.set_title('Distribution of monthly charges by churn')

In [ ]:
corr = df_pandas.apply(lambda x: pd.factorize(x)[0]).corr()
mask = np.triu(np.ones_like(corr, dtype=bool))

heat = go.Heatmap(
    z=corr.mask(mask),
    x=corr.columns,
    y=corr.columns,
    colorscale=px.colors.diverging.RdBu,
    zmin=-1,
    zmax=1
)

pio.templates.default = "plotly_white"


fig.update_xaxes(side="bottom")

fig.update_layout(
    title_text='Heatmap', 
    title_x=0.5, 
    width=1000, 
    height=1000,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    yaxis_autorange='reversed',
    template='plotly_white'
)

fig=go.Figure(data=[heat])
fig.show()

## Limpieza de datos

In [ ]:
df.names

In [ ]:
df.stypes

In [ ]:
##missing values
dt.math.isna(df).sum()

We only have 11 missing values in the TotalCharges column. 

In [ ]:
##Delete missing rows
df = df[dt.rowall(dt.f[:] != None), :]

In [ ]:
#Delete customerID
del df[:, "customerID"]

In [ ]:
df.head()

In [ ]:
#Enconde Churn
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df[:, 'Churn'] = dt.Frame(le.fit_transform(np.ravel(df[:, 'Churn'])))

In [ ]:
# Function for OHE
def ohe_columns(columns,df):
    df_work = df.copy()
    for column in columns:
        df_ohe = dt.str.split_into_nhot(df_work[column])
        df_ohe.names = [f'{column}_{col}' for col in df_ohe.names]
        df_work.cbind(df_ohe)
    return df_work

In [ ]:
# Select categorical columns
categorical_columns = df[:, str].names

In [ ]:
# Get final df after OHE
df_final = ohe_columns(categorical_columns,df)

In [ ]:
# Delete original columns
del df_final[:, categorical_columns]

In [ ]:
df_final.head()

In [ ]:
df_final.to_csv("../data/churn_clean.csv")

## Modelado

### H2O

In [ ]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o --user

In [ ]:
import h2o

In [ ]:
h2o.init()

In [ ]:
dataset = h2o.H2OFrame(df_final.to_pandas())

In [ ]:
dataset.head()

### 2. GBM with H2O

In [ ]:
from h2o.estimators import *
from h2o.grid import *

In [ ]:
train, valid, test = dataset.split_frame([0.7, 0.15], seed=42)

In [ ]:
# Identify predictors and response
x = train.columns
y = "Churn"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()
valid[y] = valid[y].asfactor()

In [ ]:
gbm = H2OGradientBoostingEstimator(seed = 42, 
                                   model_id = 'default_gbm')

%time gbm.train(x = x, y = y, training_frame = train, validation_frame = valid)

In [ ]:
gbm

In [ ]:
gbm.predict(valid)

In [ ]:
default_gbm_per = gbm.model_performance(test)

In [ ]:
default_gbm_per

In [ ]:
# Hyperparameter estimation

gbm = H2OGradientBoostingEstimator(ntrees = 500,
                                   learn_rate = 0.05,
                                   seed = 42,
                                   model_id = 'grid_gbm')

hyper_params_tune = {'max_depth' : [4, 5, 6, 7, 8],
                     'sample_rate': [x/100. for x in range(20,101)],
                     'col_sample_rate' : [x/100. for x in range(20,101)],
                     'col_sample_rate_per_tree': [x/100. for x in range(20,101)],
                     'col_sample_rate_change_per_level': [x/100. for x in range(90,111)]}

search_criteria_tune = {'strategy': "RandomDiscrete",
                        'max_runtime_secs': 90,  
                        'max_models': 100,  ## build no more than 100 models
                        'seed' : 42}

random_grid = H2OGridSearch(model = gbm, hyper_params = hyper_params_tune,
                            grid_id = 'random_grid',
                            search_criteria = search_criteria_tune)

%time random_grid.train(x = x, y = y, training_frame = train, validation_frame = valid)

In [ ]:
sorted_random_search = random_grid.get_grid(sort_by = 'auc',decreasing = True)
sorted_random_search.sorted_metric_table()

In [ ]:
tuned_gbm = sorted_random_search.models[0]

In [ ]:
tuned_gbm_per = tuned_gbm.model_performance(test)
print(tuned_gbm_per.auc())

In [ ]:
tuned_gbm.explain(test)

In [ ]:
tuned_gbm.explain_row(test, row_index=0)